In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ucimlrepo import fetch_ucirepo 
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
import category_encoders as ce
import scorecardpy as sc

In [ ]:
%pip install category_encoders

In [ ]:
# считываем данные и сохраняем их в датафрейм
df = pd.read_csv("/Users/artemshevchenko/Downloads/Проект по скорингу 4/tr_for_students.csv", sep=",")

val = pd.read_csv("/Users/artemshevchenko/Downloads/Проект по скорингу 4/vl_for_students.csv", sep=",")

#  разделим наш тренировочный датасет на две части - в одном будут содержаться численные переменные, а в другом строковые переменные и прочие объекты
continious = df.select_dtypes(include=['float', 'int'])
categorical = df.select_dtypes(include=['object'])

# будем всегда иметь под рукой расшифровку названий колонок, поскольку лазать за ними в эксель каждый раз очень неудобно
col_desk = pd.read_excel("/Users/artemshevchenko/Downloads/Проект по скорингу 4/Columns description.xlsx", usecols=[1, 2])


In [ ]:
print("Размер тренировочной выборки:")
print(df.shape)
print()
print("Размер валидационной выборки:")
print(val.shape)

## Распределение непрерывных переменных

## Распределение категориальных переменных:

## Data Preprocessing

Сразу оговоримся, что так как пропущенные значения есть и в тренировочной, и валидационной выборке, заполнять мы будем их обе. 

Для начала почистим наш датафрейм от неинформативных признаков. Так, например, почти точно признаки issue_d, содержащий дату, а также признак addr_state, содержащий локацию заявителя, не информативны для датасета.

In [ ]:
df.drop('issue_d', axis=1, inplace=True)
# df.drop('addr_state', axis=1, inplace=True)

val.drop('issue_d', axis=1, inplace=True)
# val.drop('addr_state', axis=1, inplace=True)

# я хотел удалить признак addr_state, однако позже протестировал гипотезу, что с его помощью
# модель работает точнее, так что он остался, а его обработка будет дальше

Заменим subgrade на числа (рейтинг)

In [ ]:
print("Тренировочная выборка:")
print(*sorted(df["sub_grade"].unique()))

print()
print("Валидационная выборка:")
print(*sorted(val["sub_grade"].unique()))

In [ ]:
from category_encoders import OrdinalEncoder

encoder = OrdinalEncoder()
df['sub_grade'] = encoder.fit_transform(df['sub_grade'].values.reshape(-1, 1))

# Преобразуйте тестовый датасет
val['sub_grade'] = encoder.transform(val['sub_grade'].values.reshape(-1, 1))

print("Для тренировочной выборки:")
print("Уникальных кредитных рейтингов", len(df["sub_grade"].unique()))
print("Пропущенных значений", df["sub_grade"].isna().sum())

print()

print("Для валидационной выборки:")
print("Уникальных кредитных рейтингов", len(val["sub_grade"].unique()))
print("Пропущенных значений", val["sub_grade"].isna().sum())

Причины, почему клиенты обращаются за кредитом, достаточно содержательны, и мы оставим их для дальнейшего анализа (purpose). Для этого заменим их на фиктивные переменные:

In [ ]:
encoder = ce.OrdinalEncoder(cols=["purpose"])
df = encoder.fit_transform(df)
val = encoder.fit_transform(val)

print("Для тренировочной выборки:")
print("Уникальных причин обращения", len(df["purpose"].unique()))
print("Пропущенных значений", df["purpose"].isna().sum())

print()

print("Для валидационной выборки:")
print("Уникальных причин обращения", len(val["purpose"].unique()))
print("Пропущенных значений", val["purpose"].isna().sum())

Посмотрим на то, насколько часто кредит выдают в зависимости от статуса владения недвижимостью (home_ownership).

In [ ]:
results = {}
df_target_temp = df["def"]

for ownership in df["home_ownership"].unique():
    result = df_target_temp[df["home_ownership"] == ownership].sum()
    results[ownership] = result

# Выведите результаты
for status in results:
    print(f"{status}: {results[status]}")

In [ ]:
encoder = ce.OrdinalEncoder(cols=["home_ownership"])

print("В тренировочной выборке")
df = encoder.fit_transform(df)
print(*df["home_ownership"].unique())

print()

print("В валидационной выборке")
val = encoder.fit_transform(val)
print(*val["home_ownership"].unique())

Как и обещал, аналогичные действия проворачиваю для признака addr_state, означаюего state, который фигурирует в заявке на кредит клиента:

In [ ]:
encoder = ce.OrdinalEncoder(cols=["addr_state"])

print("В тренировочной выборке")
df = encoder.fit_transform(df)
print(*df["addr_state"].unique())

print()

print("В валидационной выборке")
val = encoder.fit_transform(val)
print(*val["addr_state"].unique())

Осталось рассмотреть признак emp_title. Тут мы уже немного затронем логику заполнения пропущенных значений в датасете, которой посвящена следующая часть проекта.

emp_title, то есть бренд работодателя - нередко может хоть и грубо, но говорить о статусе и влиять на кредитоспособность клиента. Мы могли бы преобразовать категориальный признак emp_title в булевый, где мы бы хранили статус работоустроенности (ведь, очевидно, пропущенные значения в этом ключе почти точно говорят об отсутствии у заявителя работы на момент подачи), однако, исходя из выше сказанного, мы поступим иначе.
Мы построим биекцию из различных названий работы в натуральные числа, а пропущенные значения заполним нулями. Так, мы и одновременно сохраним информативность признака по наличию работы у заявителя, и, возможно, проследим связь между решение о выдаче кредита разным заявителем, работающим в одном и том же месте.

Для начала посмотрим, сколько всего уникальных работодателей указано в датасет. В работе мы полагаем, что человеческий фактор в виде допущения ошибки при указании работы в анкете (APPLE и APLE), из-за чего названия будут зашифрованы в разные натуральные числа, незначителен и не повлияет на точность модели: 

In [ ]:
# заполним пропущенные значения при отсутствии работы нулём
df["emp_title"].fillna(0, inplace=True)
val["emp_title"].isna().sum()

In [ ]:
print("В тренировочной выборке:")
print("Всего заполненных полей про работу", df["emp_title"].shape[0])
print("Уникальных работодателей в датасете", df["emp_title"].unique().shape[0])

print()

print("В валидационной выборке:")
print("Всего заполненных полей про работу", val["emp_title"].shape[0])
print("Уникальных работодателей в датасете", val["emp_title"].unique().shape[0])

Так, видно, что ожидание большого количества совпадащих работодателей крайне мало, однако мы поступим так, как договорились:

In [ ]:
encoder = ce.OrdinalEncoder(cols=["emp_title"])

df = encoder.fit_transform(df)
val = encoder.fit_transform(val)

print("В тренировочной выборке:")
print(*df["emp_title"].unique())

print()

print("В валидационной выборке:")
print(*val["emp_title"].unique())

Теперь замёмся пропущенными значениями, а именно переменными типа NaN. Определим сколько их, в каких столбцах они содержатся, далее заполним их по определённой логике

# NaN

In [ ]:
# поймём, сколько в нашем датафрейме переменных типа NaN
pd.DataFrame(df.isna().sum())

In [ ]:
pd.DataFrame(val.isna().sum())

Посмотрим (по тренировочной выборке), есть ли в пропущенных значениях emp_length такие, которые мы могли бы предсказать по старому имеющемся набору пар (emp_title, emp_length)

In [ ]:
# посмотрим, скольким людям из тех, чье значения переменной emp_length не указаны, одобряли кредит, а каким - нет
print("Люди, чье значения переменной emp_length не указаны, но кому одобряли кредит:")
print(df_target_temp[df["emp_length"].isna()].sum())

# их всего 531 из 61147, так что этими значениями можно пренебречь. однако мы посмотрим, у кого из них указана должность, а у кого - нет
# посмотрим, сколько тех, у кого указана должность, emp_length = 0, а df_target = 1

needed_indexes = df_target_temp[(df["emp_length"].isna()) & (df["emp_title"] != 0)].index
print()
print("Люди, у которых указана должность, emp_length = 0, а df_target = 1")
print(df_target_temp[needed_indexes].sum())

# имеем, что у 0 из 531 человек, у которых не указан emp_length, указана должность. поэтому мы можем смело заменить все NaN на 0
# имеем, что у всех 531 человек, у которых emp_length не указан, есть название работы. тогда просто отбросить эти значения мы не можем, так что по-хорошему заполнить 
# их средний временем работы

df["emp_length"].fillna(df["emp_length"].mean(), inplace=True)
val["emp_length"].fillna(val["emp_length"].mean(), inplace=True)

# проверим, что NaN значений больше нет
print()
print("Пропущенных значений в тренировочной выборке:")
print(df["emp_length"].isna().sum())
print()
print("Пропущенных значений в валидационной выборке:")
print(val["emp_length"].isna().sum())

Разберёмся с mths_since_recent_inq, num_accts_ever_120_pd, num_tl_90g_dpd_24m, acc_open_past_24mths, avg_cur_bal, tot_hi_cred_lim:

Заметим, что mths_since_recent_inq (Месяцы с последнего запроса), m_accts_ever_120_pd (Количество счетов более 120 дней просроченных за всю историю), num_tl_90g_dpd_24m (Количество счетов с просрочкой более 90 дней за последние 24 месяца), avg_cur_bal (Средний текущий баланс по всем счетам), tot_hi_cred_lim (Общий кредитный лимит), acc_open_past_24mths (Количество открытых счетов за последние 24 месяца) - все имеют достаточно схожее количество пропущенных значений в переменных. 

In [ ]:
pd.DataFrame(df.isna().sum())

In [ ]:
pd.DataFrame(val.isna().sum())

При этом важно, что все эти признаки содержат в своих переменных нулевые значения:

In [ ]:
def contains_null(feature):
    if (0, 0) != df[df[f"{feature}"] == 0].shape:
        print(f"{feature} contains null values")
    else:
        print(f"{feature} does not contain null values")

contains_null("mths_since_recent_inq")
contains_null("num_accts_ever_120_pd")
contains_null("num_tl_90g_dpd_24m")
contains_null("avg_cur_bal")
contains_null("tot_hi_cred_lim")
contains_null("acc_open_past_24mths")

Таким образом, заполнять данные значения нулями создаст потенциальную неточность ввиду достаточно большого количества пропущенных значений. С другой стороны, логическое преобразование признака в другой, более информативный, также представляется трудной задачей - 

Попробуем рассмотреть эти признаки по отдельности и совершим логические преобразования, разделив тех, у кого прежде просто не было счетов и тех, кто имел счет, но не делал по нему просрочек по выплатам, не создавал запросов на новые кредиты или имел нулевой кредитный лимит:

Для примера, рассмотрим mths_since_recent_inq:

$$\begin{cases}
    \text{mthssincerecentinq[i] } = 0 \text{ - запрашивал в течение последнего месяца} \\
    \text{mthssincerecentinq[i]  = NaN - не запрашивал вообще}
\end{cases}$$
$$\Downarrow $$
$$\begin{cases}
    \text{запрашивал когда-либо} \Rightarrow \text{mthssincerecentinq[i]  } += 1 \\
    \text{запрашивал в течение последнего месяца} \Rightarrow \text{mthssincerecentinq[i]  } = 1 \\
    \text{не запрашивал вообще} \Rightarrow \text{mthssincerecentinq[i]  } = 0
\end{cases}$$

Воспользуемся той же логикой для заполнения пропущенных значений в выше указанных признаках:

In [ ]:
increase_numeric = lambda x: x + 1 if pd.notna(x) else 0

for feature in ["mths_since_recent_inq", "num_accts_ever_120_pd", "num_tl_90g_dpd_24m", "avg_cur_bal", "tot_hi_cred_lim", "acc_open_past_24mths"]:
    df[f"{feature}"] = df[f"{feature}"].apply(increase_numeric)
    val[f"{feature}"] = val[f"{feature}"].apply(increase_numeric)


Посмотрим, остались ли пропущенные значения в столбцах:

In [ ]:
pd.DataFrame(df.isna().sum())

In [ ]:
pd.DataFrame(val.isna().sum())

## Генерация новых признаков:


Попробуем сгенерировать ряд новых признаков из имеющихся:

## EDA

In [ ]:
df

In [ ]:
val

In [ ]:
df_target = df["def"]
del df["def"]

val_target = val["def"]
del val["def"]

df_for_eda = pd.concat([df, val], ignore_index=True)

print("Размер тренировочной выборки:")
print(df.shape)
print()
print("Размер валидационной выборки:")
print(val.shape)


In [ ]:
non_numeric_columns = df_for_eda.select_dtypes(exclude=[np.number]).columns

# Вывод результатов
if len(non_numeric_columns) == 0:
    print("df_for_eda содержит только числа.")
else:
    print("df_for_eda содержит нечисловые значения в следующих столбцах:")
    print(non_numeric_columns)

In [ ]:
fig_cor = px.imshow(df_for_eda.corr(), text_auto=True)
fig_cor.update_layout(
     title={
        "text": "Корреляция переменных",
        "x": 0.5
    },

)
fig_cor.show()

In [ ]:
X_train, y_train, X_test, y_test = df, df_target, val, val_target

## Линейная регрессия:

In [ ]:
# создание модели линейной регрессии
model = LinearRegression()

# обучение модели
model.fit(X_train, y_train)

# получение прогнозов
y_pred = model.predict(X_test)

# оценка качества модели
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# вывод результатов
print(f"Среднеквадратичная ошибка (MSE): {mse}")
print(f"Коэффициент детерминации (R^2): {r2}")

print("")

# получение коэффициентов регрессии
coefficients = pd.DataFrame({'Признак': X_train.columns, 'Коэффициент': model.coef_})
print(coefficients)

In [ ]:
threshold = 0.5

# создание массива с замененными значениями
y_pred_new = np.where(y_pred < threshold, 0, 1)

# посмотрим срез из 5 элементов в старом и новом массивах, а также на типы данных, содержащихся в них
print(y_pred[:5])
print(y_pred.dtype)
print("")
print(y_pred_new[:5])
print(y_pred_new.dtype)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_pred_new)

warnings.filterwarnings("ignore")
fig_mis = px.imshow(cm, text_auto=True)
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()
warnings.filterwarnings("default")

## Логистическая регрессия

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression

# # доступные solver-ы для модели логистической регрессии в sklearn
# solvers = ['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga']
# # возможные значения достаточного количество итераций для сходимости модели
# max_iters = [10, 100, 500, 1000, 5000, 5100, 10000] 

# # некоторые переменные для удобства
# best_accuracy = 0
# best_solver = None
# best_max_iter = None

# # игнорируем предупреждения о недостаточном количестве итераций
# warnings.filterwarnings("ignore")

# for solver in solvers:
#     for max_iter in max_iters:
#         model = LogisticRegression(solver=solver, max_iter=max_iter)
#         model.fit(X_train, y_train)
#         y_pred = model.predict(X_test)
#         accuracy = accuracy_score(y_test, y_pred)

#         if accuracy > best_accuracy:
#             best_accuracy = accuracy    
#             best_solver = solver
#             best_max_iter = max_iter
            
# # восстанавливаем вывод предупреждений
# warnings.filterwarnings("default")

# print("Лучший solver:", best_solver)
# print("Лучшее количество итераций (max_iter):", best_max_iter)
# print("Лучшая accuracy:", best_accuracy)


In [ ]:
model = LogisticRegression(solver="lbfgs", max_iter=5000)
iterations = []
accuracies = []

max_iter_values = [10, 20, 30, 40, 50, 100, 200, 500, 1000]

warnings.filterwarnings("ignore")

# обучение модели и запись точности
for max_iter in max_iter_values:
    model.max_iter = max_iter
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    iterations.append(max_iter)
    accuracies.append(accuracy)
    
warnings.filterwarnings("default")

plt.figure(figsize=(10, 6))
plt.plot(iterations, accuracies, marker='o')
plt.title('Зависимость точности от количества итераций')
plt.xlabel('Количество итераций')
plt.ylabel('Точность')
plt.grid(True)
plt.show()


In [ ]:
# создание модели логистической регрессии
model = LogisticRegression(solver = "liblinear", max_iter = 500)

# обучение модели
model.fit(X_train, y_train)

# получение прогнозов
y_pred = model.predict(X_test)

# оценка качества модели
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

warnings.filterwarnings("ignore")
fig_mis = px.imshow(cm, text_auto=True)
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()
warnings.filterwarnings("default")


In [ ]:
print('accuracy_score= {:.3f}'.format(accuracy_score(y_test, y_pred)))
print('recall_score = {:.3f}'.format(recall_score(y_test, y_pred)))
print('precision_score = {:.3f}'.format(precision_score(y_test, y_pred)))

## Решающие деревья

In [ ]:
rfc = RandomForestClassifier(random_state=2)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

print('Model accuracy: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

warnings.filterwarnings("ignore")
fig_mis = px.imshow(cm, text_auto=True)
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()
warnings.filterwarnings("default")

In [ ]:
print('accuracy_score= {:.3f}'.format(accuracy_score(y_test, y_pred)))
print('recall_score = {:.3f}'.format(recall_score(y_test, y_pred)))
print('precision_score = {:.3f}'.format(precision_score(y_test, y_pred)))

## ROC-AUC

In [ ]:
lr_probs = model.predict_proba(X_train)[:, 1]
lr_probs_val = model.predict_proba(X_test)[:, 1]

In [ ]:

train_roc = sc.perf_eva(y_train, lr_probs, plot_type=["roc"], title="train")
val_roc = sc.perf_eva(y_test,lr_probs_val, plot_type=["roc"], title="test")

Результаты ROC-AUC получились не очень удовлетворительными, но я ещё не раз буду проводить работу над обработкой данных с соответвующими комментариями.